<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/conv_specification_example/tests/conv_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00


In [26]:
# @title Standard convolution (1 channel)
import numpy
from onnx import *

# Create inputs
x = make_tensor_value_info('x', TensorProto.FLOAT, [1, 1, 8, 8])
w = make_tensor_value_info('w', TensorProto.FLOAT, [1, 1, 3, 2])
b = make_tensor_value_info('b', TensorProto.FLOAT, [1, 1])

# Create a node (Conv) with input/outputs
node_def = make_node(
    'Conv', # node name
    ['x', 'w', 'b'], # inputs
    ['y'], # outputs
    dilations=[2,2],
    kernel_shape=[3,2],
    pads=[1, 2, 2, 2],
    strides=[2, 3],
    auto_pad='NOTSET',
    group=1, # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-conv',
    [x, w, b],
    [helper.make_tensor_value_info('y', TensorProto.FLOAT, [1, 1, 4, 4])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors
x = numpy.ones((1, 1, 8, 8), dtype=numpy.float32)
w = numpy.ones((1, 1, 3, 2), dtype=numpy.float32)
b = numpy.ones((1, 1), dtype=numpy.float32)

y = sess.run(None, {'x': x, 'w':w, 'b': b})[0]

print("X shape:", x.shape)
print("X:", x)

print("W shape:", w.shape)
print("W:", w)

print("B shape:",
      b.shape)
print("B:", b)

print("Y shape:", y.shape)
print("Y:", y)


graph test-conv (
  %x[FLOAT, 1x1x8x8]
  %w[FLOAT, 1x1x3x2]
  %b[FLOAT, 1x1]
) {
  %y = Conv[auto_pad = 'NOTSET', dilations = [2, 2], group = 1, kernel_shape = [3, 2], pads = [1, 2, 2, 2], strides = [2, 3]](%x, %w, %b)
  return %y
}
X shape: (1, 1, 8, 8)
X: [[[[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]]]
W shape: (1, 1, 3, 2)
W: [[[[1. 1.]
   [1. 1.]
   [1. 1.]]]]
B shape: (1, 1)
B: [[1.]]
Y shape: (1, 1, 4, 4)
Y: [[[[3. 5. 5. 3.]
   [4. 7. 7. 4.]
   [4. 7. 7. 4.]
   [3. 5. 5. 3.]]]]


In [44]:
# @title Standard convolution (3 channels)
import numpy
from onnx import *

# Create inputs
x = make_tensor_value_info('x', TensorProto.FLOAT, [1, 3, 8, 8])
w = make_tensor_value_info('w', TensorProto.FLOAT, [1, 3, 3, 2])
b = make_tensor_value_info('b', TensorProto.FLOAT, [1, 1])

# Create a node (Conv) with input/outputs
node_def = make_node(
    'Conv', # node name
    ['x', 'w', 'b'], # inputs
    ['y'], # outputs
    dilations=[2,2],
    kernel_shape=[3,2],
    pads=[1, 2, 2, 2],
    strides=[2, 3],
    auto_pad='NOTSET',
    group=1, # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-conv',
    [x, w, b],
    [helper.make_tensor_value_info('y', TensorProto.FLOAT, [1, 1, 4, 4])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors
x = numpy.ones((1, 3, 8, 8), dtype=numpy.float32)
w = numpy.ones((1, 3, 3, 2), dtype=numpy.float32)
b = numpy.ones((1, 1), dtype=numpy.float32)

y = sess.run(None, {'x': x, 'w':w, 'b': b})[0]

print("X shape:", x.shape)
print("X:", x)

print("W shape:", w.shape)
print("W:", w)

print("B shape:",
      b.shape)
print("B:", b)

print("Y shape:", y.shape)
print("Y:", y)


graph test-conv (
  %x[FLOAT, 1x3x8x8]
  %w[FLOAT, 1x3x3x2]
  %b[FLOAT, 1x1]
) {
  %y = Conv[auto_pad = 'NOTSET', dilations = [2, 2], group = 1, kernel_shape = [3, 2], pads = [1, 2, 2, 2], strides = [2, 3]](%x, %w, %b)
  return %y
}
X shape: (1, 3, 8, 8)
X: [[[[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]]]
W shape: (1, 3, 3, 2)
W: [[[[1. 1.]
   

In [42]:
# @title Depthwise convolution (3 channels)

import numpy
from onnx import *

# Create inputs with updated channel size
x = make_tensor_value_info('x', TensorProto.FLOAT, [1, 3, 8, 8])  # Changed channels from 1 to 3
w = make_tensor_value_info('w', TensorProto.FLOAT, [3, 1, 3, 2])  # Updated to match input channels
b = make_tensor_value_info('b', TensorProto.FLOAT, [3])           # Bias shape matches output channels

# Create a node (Conv) with input/outputs
node_def = make_node(
    'Conv',  # node name
    ['x', 'w', 'b'],  # inputs
    ['y'],  # outputs
    dilations=[2, 2],
    kernel_shape=[3, 2],
    pads=[1, 2, 2, 2],
    strides=[2, 3],
    auto_pad='NOTSET',
    group=3,  # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-conv',
    [x, w, b],
    [helper.make_tensor_value_info('y', TensorProto.FLOAT, [1, 1, 4, 4])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors with updated channel size
x = numpy.ones((1, 3, 8, 8), dtype=numpy.float32)  # Updated to 3 channels
w = numpy.ones((3, 1, 3, 2), dtype=numpy.float32)  # Updated to match input channels
b = numpy.ones((3,), dtype=numpy.float32)          # Matches output channels

y = sess.run(None, {'x': x, 'w': w, 'b': b})[0]

print("X shape:", x.shape)
print("X:", x)

print("W shape:", w.shape)
print("W:", w)

print("B shape:", b.shape)
print("B:", b)

print("Y shape:", y.shape)
print("Y:", y)


graph test-conv (
  %x[FLOAT, 1x3x8x8]
  %w[FLOAT, 3x1x3x2]
  %b[FLOAT, 3]
) {
  %y = Conv[auto_pad = 'NOTSET', dilations = [2, 2], group = 3, kernel_shape = [3, 2], pads = [1, 2, 2, 2], strides = [2, 3]](%x, %w, %b)
  return %y
}
X shape: (1, 3, 8, 8)
X: [[[[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]]]
W shape: (3, 1, 3, 2)
W: [[[[1. 1.]
   [1

In [ ]:
from onnx import TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model
from onnxruntime import InferenceSession
import numpy


onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

check_model(onnx_model)

def shape2tuple(shape):
    return tuple(getattr(d, 'dim_value', 0) for d in shape.dim)


with open("test", "wb") as f:
    f.write(onnx_model.SerializeToString())


# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Let's see the output.
sess = InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

x = numpy.ones((1,1, 5, 5), dtype=numpy.float32)
w = numpy.ones((1,1, 3, 3), dtype=numpy.float32)
res = sess.run(None, {'x': x, 'w':w})

print("X shape:", x.shape)
print("X:", x)

print("W shape:", w.shape)
print("W:", w)

print(res[0])
y=res[0]

print("Y shape:", y.shape)
print("Y:", y)


graph test-model (
  %x[FLOAT, 1x1x8x8]
  %w[FLOAT, 1x1x3x2]
) {
  %y = Conv[dilations = [1, 2], group = 1, pads = [1, 2, 2, 2], strides = [2, 3]](%x, %w)
  return %y
}


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: x for the following indices
 index: 2 Got: 5 Expected: 8
 index: 3 Got: 5 Expected: 8
 Please fix either the inputs/outputs or the model.

In [ ]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import numpy as np
import onnxruntime as ort

class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        # Set custom kernel values
        custom_kernel = torch.tensor([[[
            [1.0, 2.0, 3.0],
            [1.0, 1.0, 1.0],
            [3.0, 2.0, 1.0]
        ]]])
        self.conv.weight = nn.Parameter(custom_kernel)


    def forward(self, x):
        return self.conv(x)

# Create an instance of the model
model = SimpleConvNet()

# Create a dummy input
dummy_input = torch.randn(1, 1, 5, 5)

# Export the model to ONNX
torch.onnx.export(model, dummy_input, "simple_conv_net.onnx", opset_version=11,
                  input_names=['input'], output_names=['output'])

# Load the ONNX model
onnx_model = onnx.load("simple_conv_net.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Create an ONNX Runtime session
session = ort.InferenceSession("simple_conv_net.onnx")

# Prepare input data
X = session.get_inputs()[0].name

X = np.array([[[[ 1,  2,  3,  4 , 5],
  [ 1,  2,  3,  4 , 5],
  [ 1,  2,  3,  4 , 5],
 [ 1,  2,  3,  4 , 5],
  [ 1,  2,  3,  4 , 5]]]]).astype(np.float32)


# Run inference
Y = session.run(None, {input_name: X})

print("Input shape:", X.shape)
print("Input:", X)

print("Output shape:", Y[0].shape)
print("Output:", Y[0])

graph main_graph (
  %input[FLOAT, 1x1x5x5]
) initializers (
  %conv.weight[FLOAT, 1x1x3x3]
  %conv.bias[FLOAT, 1]
) {
  %output = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%input, %conv.weight, %conv.bias)
  return %output
}
Input shape: (1, 1, 5, 5)
Input: [[[[1. 2. 3. 4. 5.]
   [1. 2. 3. 4. 5.]
   [1. 2. 3. 4. 5.]
   [1. 2. 3. 4. 5.]
   [1. 2. 3. 4. 5.]]]]
Output shape: (1, 1, 5, 5)
Output: [[[[ 7.269953 16.269953 25.269953 34.269955 31.269953]
   [15.269953 30.269953 45.269955 60.269955 45.269955]
   [15.269953 30.269953 45.269955 60.269955 45.269955]
   [15.269953 30.269953 45.269955 60.269955 45.269955]
   [11.269953 20.269953 29.269953 38.269955 23.269953]]]]
